In [ ]:
from bs4 import BeautifulSoup
import csv
import chardet

def limpiar_texto(texto):
    return (
        texto
        .replace("–", "-")
        .replace("—", "-")
        .replace("“", '"')
        .replace("”", '"')
        .replace("\xa0", " ")
        .strip()
    )

# Detectar codificación
with open("CatalogoGeneral.htm", "rb") as f:
    raw_data = f.read()
    encoding_detectado = chardet.detect(raw_data)["encoding"]
    print(f"Codificación detectada: {encoding_detectado}")

# Decodificar el contenido correctamente
html_text = raw_data.decode(encoding_detectado, errors="replace")
soup = BeautifulSoup(html_text, "html.parser")

# Seleccionar tabla del catálogo
tables = soup.find_all("table")
catalogo_table = tables[1]  # Ajusta si cambia

# Filas de datos
rows = catalogo_table.find_all("tr")

# Encabezados
columnas = [
    "Compositor", "Obra", "Año", "Duración",
    "Género y efectivo", "Estreno", "Edición discográfica",
    "Notas", "Grabación 1"
]

# Guardar CSV
with open("catalogo_inicial.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(columnas)

    for row in rows[1:]:
        cells = row.find_all("td")
        cell_texts = []

        for cell in cells:
            content = " ".join([p.get_text(strip=True) for p in cell.find_all("p")]) or cell.get_text(strip=True)
            content = limpiar_texto(content)
            cell_texts.append(content)

        if len(cell_texts) >= 5:
            fila = (cell_texts + [""] * 10)[:10]
            writer.writerow(fila)